In [4]:
class BTreeNode(object):
    def __init__(self, keys):
        self.keys = keys
        self.children = []

    def add_child(self, child_keyss):
        self.children.append(BTreeNode(child_keyss))
        self.children.sort()

    def add_key(self, key):
        self.keys.append(key)
        self.keys.sort()

    def remove_key(self, key):
        self.keys.remove(key)

    def __lt__(self, other):
        return self.keys[-1] < other.keys[-1]

    def draw(self, root_node=True):
        my_str=''
        if root_node:
            my_str = '''
                        <!DOCTYPE html>
                    <html lang="en">
                      <head>
                        <meta charset="utf-8">

                        <title>Collapsible Tree Example</title>

                        <style>

                        .node circle {
                          fill: #fff;
                          stroke: steelblue;
						  stroke-width: 3px;
						}

						.node text { font: 12px sans-serif; }

						.link {
						  fill: none;
						  stroke: #ccc;
						  stroke-width: 2px;
						}
						
					    </style>

					  </head>

					  <body>

					<!-- load the d3.js library -->	
					<script src="https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.17/d3.min.js"></script>
						
					<script>

					var treeData = [
					  '''
	
        my_str += '{"name": "'
        for k in self.keys:
            my_str += str(k) + ', '
        my_str = my_str[:-2]
        my_str += '" '

        if len(self.children) > 0:
            my_str += ', "children": ['

        value_id = 0		
        for child in self.children:
            my_str += child.draw(root_node=False) + ', '

        if len(self.children) > 0:
            my_str += ']'

        my_str += '}'

        if root_node:
            my_str += '''
			];

			// ************** Generate the tree diagram	 *****************
			var margin = {top: 40, right: 120, bottom: 20, left: 120},
				width = 1500 - margin.right - margin.left,
				height = 800 - margin.top - margin.bottom;
				
			var i = 0;

			var tree = d3.layout.tree()
				.size([height, width]);

			var diagonal = d3.svg.diagonal()
				.projection(function(d) { return [d.x, d.y]; });

			var svg = d3.select("body").append("svg")
				.attr("width", width + margin.right + margin.left)
				.attr("height", height + margin.top + margin.bottom)
			  .append("g")
				.attr("transform", "translate(" + margin.left + "," + margin.top + ")");

			root = treeData[0];
			  
			update(root);

			function update(source) {

			  // Compute the new tree layout.
			  var nodes = tree.nodes(root).reverse(),
				  links = tree.links(nodes);

			  // Normalize for fixed-depth.
			  nodes.forEach(function(d) { d.y = d.depth * 100; });

			  // Declare the nodes…
			  var node = svg.selectAll("g.node")
				  .data(nodes, function(d) { return d.id || (d.id = ++i); });

			  // Enter the nodes.
			  var nodeEnter = node.enter().append("g")
				  .attr("class", "node")
				  .attr("transform", function(d) { 
					  return "translate(" + d.x + "," + d.y + ")"; });

			  nodeEnter.append("circle")
				  .attr("r", 10)
				  .style("fill", "#fff");

			  nodeEnter.append("text")
				  .attr("y", function(d) { 
					  return d.children || d._children ? -18 : 18; })
				  .attr("dy", ".35em")
				  .attr("text-anchor", "middle")
				  .text(function(d) { return d.name; })
				  .style("fill-opacity", 1);

			  // Declare the links…
			  var link = svg.selectAll("path.link")
				  .data(links, function(d) { return d.target.id; });

			  // Enter the links.
			  link.enter().insert("path", "g")
				  .attr("class", "link")
				  .attr("d", diagonal);

			}

			</script>
				
			  </body>
			</html>

			'''
		
            with open('tree.html', 'w+') as fp:
                fp.write(my_str)
            from IPython.display import IFrame

            return IFrame(src='./tree.html', width=1500, height=400)
        else:
            return my_str
        
   
    def search_key(self, key):
        #teste ob Schlüssel in der Liste der Schlüssel enthalten ist
        for i in range(0,len(self.key)):
            if(self.keys[i]==key):
                return print("Key schon steht")
        # Prüfe rekursiv die Schlüssel der Kinder
        if len(self.children)==0:
            return 1
        else:
            for i in range(0,len(self.keys)):
                if self.keys[i]>key:
                    return self.children[i].search_key(key)
            if self.key[len(self.keys)]<key:
                return self.children[len(self.keys)].search_key(key)
            
    def find_Position(self, key):
        #Finde Blattknoten in welcher  der Schlüssel hinzugefügt werden kann.
        if len(self.children)!=0:
            for i in range(0,len(self.keys)):
                if self.keys[i]>key:
                    return self.children[i].find_Position(key)
            if self.keys[len(self.keys)]<key:
                return self.children[len(self.keys)].find_Position(key)
        #Gebe die Liste der Schlüssel dieses Knotens aus.
        else:
            return self.keys
        
    def add_key2(self,key):
        # Für alle anderen Knoten gilt das entsprechende Blattknoten zu finden, das Element hinzuzufügen
        Flag=0                                   #wenn bei Test [73,29,27,4,3,2,31] in Baum einfügen, dann sind diese Variable Flag notwendig. 
        if len(self.children)!=0:
            for i in range(0,len(self.keys)):
                if self.keys[i]==key:
                    return self
                if self.keys[i]>key:
                    Flag=len(self.children[i].keys)
                    KinderNode=self.children[i].add_key2(key)
                    break
            if self.keys[len(self.keys)-1]<key:
                i=i+1
                Flag=len(self.children[i].keys)
                KinderNode=self.children[i].add_key2(key)
        #Gebe die Liste der Schlüssel dieses Knotens aus.
        else:
            self.add_key(key)
            if len(self.keys)>3:
                self.split()
            return self
        # und wenn notwendig zu splitten
        if len(KinderNode.keys)>1 or (Flag==1):
            self.children[i]=KinderNode
            self.children.sort()
        else:
            self.children[i]=KinderNode.children[0]
            self.children.append(KinderNode.children[1])
            self.children.sort()
            self.add_key(KinderNode.keys[0])
            if len(self.keys)>3:
                supNode=BTreeNode([self.keys[0],self.keys[1]])
                supNode.children.append(self.children[0])
                supNode.children.append(self.children[1])
                supNode.children.append(self.children[2])
                supNode2=BTreeNode([self.keys[3]])
                supNode2.children.append(self.children[3])
                supNode2.children.append(self.children[4])
                self.children.clear()
                self.keys.remove(self.keys[3])
                self.keys.remove(self.keys[1])
                self.keys.remove(self.keys[0])
                self.children.append(supNode)
                self.children.append(supNode2)
        return self

    def split(self):
        #Erstellen von zwei Knoten 
        #Verteilung der Schlüssel und Kinder
        self.add_child([self.keys[3]])
        self.add_child([self.keys[0],self.keys[1]])
        self.remove_key(self.keys[3])
        self.remove_key(self.keys[1])
        self.remove_key(self.keys[0])
        
        

In [5]:
tree = BTreeNode([44])
tree.add_child([15, 22, 30])
tree.add_child([55, 60, 77])

tree.children[0].add_child([1,5,10])
tree.children[0].add_child([17])
tree.children[0].add_child([25,28])
tree.children[0].add_child([39,41])

tree.children[1].add_child([47, 51])
tree.children[1].add_child([56, 58])
tree.children[1].add_child([66, 69, 72])
tree.children[1].add_child([79, 91])

list=[73,29,27,4,3,2]
for i in range(0,len(list)):
    tree.add_key2(list[i])
tree.draw()